In [1]:
import numpy as np

In [5]:
ts1 = np.loadtxt("ts1.txt")
ts2 = np.loadtxt("ts2.txt")
ts3 = np.loadtxt("ts3.txt")

In [4]:
from Operations.ST_LocalExtrema import ST_LocalExrema as STLE

In [1]:
from ctypes import *
import ctypes

In [7]:
so_file = "/Users/joshua/Desktop/MS_shannon.so"
lib = CDLL(so_file)

In [8]:
lib.entropy.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, flags='C_CONTIGUOUS'),
    ctypes.c_int,
    ctypes.c_int,
    ctypes.c_int,
    ctypes.POINTER(ctypes.c_double)
]
lib.entropy.restype = None  # void function

In [9]:
def shannon_entropy(data, bin_count, depth):
    """
    Calculate Shannon Entropy using the C function.
    
    Args:
    data (numpy.array): Input time series data
    bin_count (int): Number of bins for encoding
    depth (int): Depth of the encoding
    
    Returns:
    float: Calculated Shannon Entropy
    """
    data = np.ascontiguousarray(data, dtype=np.float64)
    length = len(data)
    result = ctypes.c_double()
    
    lib.entropy(data, length, bin_count, depth, ctypes.byref(result))
    
    return result.value

In [10]:
shannon_entropy(ts3, 2, 5)

3.258990526199341

In [12]:
numBins = 10
numBins = [numBins]

In [2]:
import numpy as np
from Operations.CO_HistogramAMI import CO_HistogramAMI
from scipy import stats
from PeripheryFunctions.BF_SignChange import BF_SignChange

def CO_CompareMinAMI(y, binMethod, numBins = 10):
    """
    """
    N = len(y)
    # Range of time lags to consider
    tauRange = np.arange(0, int(np.ceil(N/2))+1)
    numTaus = len(tauRange)

    # range of bin numbers to consider
    if isinstance(numBins, int):
        numBins = [numBins]
    
    numBinsRange = len(numBins)
    amiMins = np.zeros(numBinsRange)

    # Calculate automutual information
    for i in range(numBinsRange):  # vary over number of bins in histogram
        amis = np.zeros(numTaus)
        for j in range(numTaus):  # vary over time lags, tau
            amis[j] = CO_HistogramAMI(y, tauRange[j], binMethod, numBins[i])
            if (j > 1) and ((amis[j] - amis[j-1]) * (amis[j-1] - amis[j-2]) < 0):
                amiMins[i] = tauRange[j-1]
                break
        if amiMins[i] == 0:
            amiMins[i] = tauRange[-1]
    # basic statistics
    print(amis)
    out = {}
    out['min'] = np.min(amiMins)
    out['max'] = np.max(amiMins)
    out['range'] = np.ptp(amiMins)
    out['median'] = np.median(amiMins)
    out['mean'] = np.mean(amiMins)
    out['std'] = np.std(amiMins, ddof=1)
    out['nunique'] = len(np.unique(amiMins))
    out['mode'], out['modef'] = stats.mode(amiMins)
    out['modef'] = out['modef']/numBinsRange

    # converged value? 
    out['conv4'] = np.mean(amiMins[-5:])

    # look for peaks (local maxima)
    # % local maxima above 1*std from mean
    # inspired by curious result of periodic maxima for periodic signal with
    # bin size... ('quantiles', [2:80])
    diff_ami_mins = np.diff(amiMins[:-1])
    positive_diff_indices = np.where(diff_ami_mins > 0)[0]
    sign_change_indices = BF_SignChange(diff_ami_mins, 1)

    # Find the intersection of positive_diff_indices and sign_change_indices
    loc_extr = np.intersect1d(positive_diff_indices, sign_change_indices) + 1
    above_threshold_indices = np.where(amiMins > out['mean'] + out['std'])[0]
    big_loc_extr = np.intersect1d(above_threshold_indices, loc_extr)

    # Count the number of elements in big_loc_extr
    out['nlocmax'] = len(big_loc_extr)

    return out


In [8]:
from Operations.CO_CompareMinAMI import CO_CompareMinAMI as coma